# Hands-on: Implement Memory with LangChain

## Overview

In this hands-on, you will implement memory using LangChain.

For some use cases, specifically for the ones that involve multiple interactions with an LLM (for example, a chat) we may need to maintain a “history” of the conversation, which in programming terms is described as memory.

The benefit of this approach is that it’s easy to understand and implement. However, there are
two important considerations:
- The token limit of LLMs still applies. If you keep adding prompts and responses to
memory, you can quickly run out of tokens.
- Cost of tokens if you’re using a hosted instance of LLMs.

LangChain provides several types of memory to help mitigate these issues, such as:
- `ConversationBufferWindowMemory`: Keeps a list of the interactions in the conversation
over time. It only uses the last K interactions.
- `ConversationSummaryBufferMemory`: Keeps a summary of interactions, using token length
rather than number of interactions to determine when to flush interactions.
- `ConversationTokenBufferMemory`: Keeps a buffer of recent interactions in memory, and
uses token length rather than number of interactions to determine when to flush
interactions.

This notebook contains sample code for using `ConversationBufferMemory` with LLM included in watsonx.ai.
At the time of writing this lab, WML API supports integration with the most basic memory type, `ConversationBufferMemory`, which keeps the entire conversation in memory until the memory fills or until it’s explicitly cleared.

In [1]:
# Install libraries
!pip install pip install ibm-watson-machine-learning --upgrade
!pip install langchain | tail -n 1

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [2]:
# Import libraries
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/ibm_watson_machine_learning/foundation_models/extensions/langchain/llm.py:60: WatsonxLLMDeprecationWarning: ibm_watson_machine_learning.foundation_models.extensions.langchain.WatsonxLLM is deprecated and will not be supported in the future. Please import from langchain-ibm instead.
To install langchain-ibm run `pip install -U langchain-ibm`.
  _raise_watsonxllm_deprecation_warning()


In [3]:
# Set IBM Cloud API key and Project ID
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"
api_key = "YhsDtKY62OcY3yBAMRF39wEyUxdXH3dgwo6aZLIeEGCz"
project_id = "bc719555-fa95-4787-8c60-4bf93019cb02"

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [5]:
# Initialize the watsonx model

model_id = 'mistralai/mixtral-8x7b-instruct-v01'

params = {
    GenParams.DECODING_METHOD: 'greedy',
    #GenParams.TEMPERATURE: 0.2,
    #GenParams.TOP_P: 1,
    #GenParams.TOP_K: 25,
    #GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 300
}

llm_model = Model(
    model_id=model_id,
    params=params,
    credentials=creds,
    project_id=project_id
)

# In order to use Langchain, we need to instantiate Langchain extension
lc_llm_model = WatsonxLLM(model=llm_model)

print("Done initializing LLM.")

Done initializing LLM.


In [6]:
# Create the memory object
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=lc_llm_model, 
    memory = memory,
    verbose=True
)

/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [7]:
# Example of a first quesiton from the user
user_input = "From the following customer complaint, extract 3 factors that caused the customer to be unhappy. \
                            Put each factor on a new line. Customer complaint: I am writing you this statement to delete the \
                            following information on my credit report. The items I need deleted are listed in the report. \
                            I am a victim of identity theft, I demand that you remove these errors to correct my report immediately! \
                            I have reported this to the federal trade commission and have attached the federal trade commission affidavit. \
                            Now that I have given you the following information, please correct my credit report or I shall proceed with involving my attorney! \
                            Numbered list of complaints:"

#user_input="Right now I am bothered! I have attempted to be patient however it is hard to be patient when you feel that you are continually being overlooked by somebody. I think you fail to remember that \Consumer detailing organizations have expected an essential part in amassing and assessing customer credit and other data on shoppers. The XXXX XXXX  is reliant on the reasonable and precision."

# Invoke the LLM
conversation.predict(input=user_input)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: From the following customer complaint, extract 3 factors that caused the customer to be unhappy.                             Put each factor on a new line. Customer complaint: I am writing you this statement to delete the                             following information on my credit report. The items I need deleted are listed in the report.                             I am a victim of identity theft, I demand that you remove these errors to correct my report immediately!                             I have reported this to the federal trade commission and have attached the federal trade commission affidavit.                             Now that I have given you

" Based on the customer complaint, the following three factors contributed to the customer's unhappiness:\n\n1. Identity Theft: The customer has been a victim of identity theft, which has led to inaccurate information on their credit report.\n\n2. Incorrect Credit Report: The customer demands the removal of specific items from their credit report, implying that these entries are incorrect and negatively impacting their credit score.\n\n3. Potential Legal Action: The customer has threatened to involve their attorney if the credit bureau does not correct the credit report promptly, indicating their dissatisfaction with the current situation and the perceived inaction of the credit bureau."

In [8]:
# Example of a second quesiton from the user
user_input = "Does the list of complaints contain a statement about identity fraud? Provide a short answer: yes or no."
conversation.predict(input=user_input)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: From the following customer complaint, extract 3 factors that caused the customer to be unhappy.                             Put each factor on a new line. Customer complaint: I am writing you this statement to delete the                             following information on my credit report. The items I need deleted are listed in the report.                             I am a victim of identity theft, I demand that you remove these errors to correct my report immediately!                             I have reported this to the federal trade commission and have attached the federal trade commission affidavit.                             Now that I have given you 

' Yes, the list of complaints does contain a statement about identity fraud. The customer explicitly mentions being a victim of identity theft.'

In [9]:
# For debugging, print the history of the conversation
print(memory.buffer)

Human: From the following customer complaint, extract 3 factors that caused the customer to be unhappy.                             Put each factor on a new line. Customer complaint: I am writing you this statement to delete the                             following information on my credit report. The items I need deleted are listed in the report.                             I am a victim of identity theft, I demand that you remove these errors to correct my report immediately!                             I have reported this to the federal trade commission and have attached the federal trade commission affidavit.                             Now that I have given you the following information, please correct my credit report or I shall proceed with involving my attorney!                             Numbered list of complaints:
AI:  Based on the customer complaint, the following three factors contributed to the customer's unhappiness:

1. Identity Theft: The customer has been a victim 

In [10]:
# Run this if you want to clear the memory buffer
memory.clear()
print(memory.buffer)